# Imbalanced classification: Bank Loan Classification

https://keras.io/examples/structured_data/imbalanced_classification/

## Introduction

This example looks at the Personal Bank Loan Classification dataset to demonstrate how to train a classification model on data with highly imbalanced classes.

https://www.kaggle.com/code/farzadnekouei/imbalanced-personal-bank-loan-classification?select=Bank_Personal_Loan_Modelling.xlsx


## First, Vectorize the Excel Data

Before we can train a neural network, we need to **load, clean, and format the data** from Excel. This process includes:

- Reading the Excel file using pandas.
- Removing columns that do not help with prediction (e.g., ID or ZIP code).
- Separating the **features** (independent variables) from the **target** (dependent variable: did the customer accept a personal loan?).

The resulting `features` and `targets` will be used to train a classification model.


In [3]:
# Import required libraries for data processing
import pandas as pd
import numpy as np

# Load data from the Excel file, using the 'Data' sheet that contains customer information
df = pd.read_excel("Bank_Personal_Loan_Modelling.xlsx", sheet_name="Data")

# Drop ID (a unique identifier) and ZIP Code (non-numeric or high-cardinality categorical),
# which are not useful for prediction and could introduce noise
df = df.drop(columns=["ID", "ZIP Code"])

# Extract feature columns (all columns except the target) and convert to float32 for neural network input
features = df.drop(columns=["Personal Loan"]).values.astype(np.float32)

# Extract the target variable (0 = no loan, 1 = accepted loan), reshape into a 2D array
targets = df["Personal Loan"].values.astype(np.uint8).reshape(-1, 1)

# Print the shape of features and targets to confirm data formatting
print("features.shape:", features.shape)   # Expected: (5000, 11)
print("targets.shape:", targets.shape)     # Expected: (5000, 1)


features.shape: (5000, 11)
targets.shape: (5000, 1)
features.shape: (5000, 11)
targets.shape: (5000, 1)


## Prepare a Validation Set

Before training the model, we divide the data into two parts:

- **Training set (80%)**: Used to fit the model.
- **Validation set (20%)**: Used to evaluate model performance on unseen data.

This helps us detect overfitting and tune hyperparameters without touching the test set (if we had one). We use a simple 80/20 split here.


In [4]:
# Split the dataset into training and validation sets (80% train, 20% validation)
num_val_samples = int(len(features) * 0.2)  # Compute how many samples will go into the validation set

# Slice the first 80% for training
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]

# Slice the last 20% for validation
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

# Print confirmation of the split
print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


Number of training samples: 4000
Number of validation samples: 1000
Number of training samples: 4000
Number of validation samples: 1000


## Analyze Class Imbalance in the Training Targets

Many business problems involve **imbalanced datasets**, where the class of interest (e.g., accepted loans) is rare. If we ignore this, the model will likely favor the majority class and perform poorly on the minority class.

Here, we:
- Count how many samples belong to each class (0 = no loan, 1 = accepted loan).
- Print the percentage of positive class (loan accepted).
- Compute **class weights** to give more importance to the rare class during training.


In [5]:
# Count the number of examples for each class in the training set
counts = np.bincount(train_targets[:, 0])  # counts[0] = class 0 (no loan), counts[1] = class 1 (loan)

# Print the number and proportion of positive class samples
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

# Compute class weights to handle imbalance during training
# These weights will be passed to model.fit() to penalize errors on minority class more heavily
weight_for_0 = 1.0 / counts[0]  # class 0 = majority
weight_for_1 = 1.0 / counts[1]  # class 1 = minority


Number of positive samples in training data: 397 (9.93% of total)
Number of positive samples in training data: 397 (9.93% of total)


In this dataset, only ~10% of customers accepted a loan offer. A model trained without adjusting for this imbalance might predict “no” for everyone and still get 90% accuracy — but be useless for identifying actual loan accepters.

## Normalize the Data Using Training Set Statistics

Neural networks perform better when input features are **normalized** — that is, they are rescaled to have a mean of 0 and standard deviation of 1.

We normalize:
- Only using the **training data** statistics (mean and std).
- Both training and validation sets using the same parameters to avoid data leakage.

This ensures fair training and stable gradient descent.


In [6]:
# Compute the mean of each feature using only the training set
mean = np.mean(train_features, axis=0)

# Subtract the mean from training and validation sets (centering)
train_features -= mean
val_features -= mean

# Compute the standard deviation of each feature using only the training set
std = np.std(train_features, axis=0)

# Divide by standard deviation to normalize to unit variance
train_features /= std
val_features /= std


Always compute normalization statistics (mean, std) from the training set only. If you include the validation or test set, it leads to data leakage, which gives overly optimistic results.



## Build a Binary Classification Model

Now we define our neural network using the Keras `Sequential` API.

- The model has multiple **dense (fully connected) layers** with ReLU activation.
- We use **Dropout** layers to reduce overfitting by randomly "turning off" neurons during training.
- The final layer uses a **sigmoid** activation to output a probability between 0 and 1 (for binary classification).

This architecture is simple, flexible, and effective for CRM, fraud detection, and other structured data tasks.


In [9]:
import keras

# Build a sequential model for binary classification
model = keras.Sequential(
    [
        # Input layer matches the number of input features
        keras.Input(shape=train_features.shape[1:]),
        # First hidden layer with 256 units and ReLU activation
        keras.layers.Dense(256, activation="relu"),
        # Second hidden layer with 256 units
        keras.layers.Dense(256, activation="relu"),
        # Dropout layer to reduce overfitting (30% of nodes are randomly dropped)
        keras.layers.Dropout(0.3),
        # Third hidden layer
        keras.layers.Dense(256, activation="relu"),
        # Additional dropout for regularization
        keras.layers.Dropout(0.3),
        # Output layer with 1 unit (binary output) and sigmoid activation
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

# Print model architecture summary
model.summary()


Model: "sequential_1"
Model: "sequential_1"


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │           3,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)

 Total params: 134,913 (527.00 KB)
 Total params: 134,913 (527.00 KB)


 Trainable params: 134,913 (527.00 KB)
 Trainable params: 134,913 (527.00 KB)


 Non-trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)


ReLU is commonly used in hidden layers because it's fast and helps prevent vanishing gradients.
Sigmoid is used at the output for binary classification because it maps output to a probability.



## Train the Model with `class_weight` to Handle Imbalance

We now compile and train the model:

- **Loss Function:** `binary_crossentropy` is used for binary classification tasks.
- **Optimizer:** Adam with a learning rate of `1e-2` is chosen for faster convergence (can be tuned).
- **Metrics:** We track precision, recall, and confusion matrix components (FP, FN, TP, TN) to get a complete picture of performance.
- **Class Weights:** These rebalance the loss to pay more attention to underrepresented classes (positive cases in our dataset).
- **Checkpoint Callback:** Saves model weights after each epoch for reproducibility and analysis.

This setup is robust and directly relevant to real-world CRM or fraud detection use cases.


In [10]:
# Define custom metrics for detailed performance tracking
metrics = [
    keras.metrics.FalseNegatives(name="fn"),  # How many real positives were missed?
    keras.metrics.FalsePositives(name="fp"),  # How many negatives were incorrectly flagged?
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),  # TP / (TP + FP)
    keras.metrics.Recall(name="recall"),        # TP / (TP + FN)
]

# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=keras.optimizers.Adam(1e-2),
    loss="binary_crossentropy",
    metrics=metrics
)

# Create a callback to save the model at each epoch
callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.keras")]

# Define class weights to handle class imbalance (learned earlier)
class_weight = {0: weight_for_0, 1: weight_for_1}

# Train the model using batch gradient descent and class weighting
model.fit(
    train_features,
    train_targets,
    batch_size=2048,                   # Large batch for fast, stable convergence
    epochs=30,
    verbose=2,
    callbacks=callbacks,               # Save checkpoints
    validation_data=(val_features, val_targets),  # Monitor validation performance
    class_weight=class_weight          # Apply imbalance correction
)


Epoch 1/30
Epoch 1/30


2/2 - 2s - 1s/step - fn: 141.0000 - fp: 1019.0000 - loss: 3.1735e-04 - precision: 0.2008 - recall: 0.6448 - tn: 2584.0000 - tp: 256.0000 - val_fn: 1.0000 - val_fp: 326.0000 - val_loss: 0.6688 - val_precision: 0.2010 - val_recall: 0.9880 - val_tn: 591.0000 - val_tp: 82.0000
2/2 - 2s - 1s/step - fn: 141.0000 - fp: 1019.0000 - loss: 3.1735e-04 - precision: 0.2008 - recall: 0.6448 - tn: 2584.0000 - tp: 256.0000 - val_fn: 1.0000 - val_fp: 326.0000 - val_loss: 0.6688 - val_precision: 0.2010 - val_recall: 0.9880 - val_tn: 591.0000 - val_tp: 82.0000


Epoch 2/30
Epoch 2/30


2/2 - 0s - 70ms/step - fn: 11.0000 - fp: 1043.0000 - loss: 2.2853e-04 - precision: 0.2701 - recall: 0.9723 - tn: 2560.0000 - tp: 386.0000 - val_fn: 9.0000 - val_fp: 93.0000 - val_loss: 0.3016 - val_precision: 0.4431 - val_recall: 0.8916 - val_tn: 824.0000 - val_tp: 74.0000
2/2 - 0s - 70ms/step - fn: 11.0000 - fp: 1043.0000 - loss: 2.2853e-04 - precision: 0.2701 - recall: 0.9723 - tn: 2560.0000 - tp: 386.0000 - val_fn: 9.0000 - val_fp: 93.0000 - val_loss: 0.3016 - val_precision: 0.4431 - val_recall: 0.8916 - val_tn: 824.0000 - val_tp: 74.0000


Epoch 3/30
Epoch 3/30


...

2/2 - 0s - 63ms/step - fn: 11.0000 - fp: 45.0000 - loss: 2.8198e-05 - precision: 0.8956 - recall: 0.9723 - tn: 3558.0000 - tp: 386.0000 - val_fn: 5.0000 - val_fp: 24.0000 - val_loss: 0.0845 - val_precision: 0.7647 - val_recall: 0.9398 - val_tn: 893.0000 - val_tp: 78.0000
2/2 - 0s - 63ms/step - fn: 11.0000 - fp: 45.0000 - loss: 2.8198e-05 - precision: 0.8956 - recall: 0.9723 - tn: 3558.0000 - tp: 386.0000 - val_fn: 5.0000 - val_fp: 24.0000 - val_loss: 0.0845 - val_precision: 0.7647 - val_recall: 0.9398 - val_tn: 893.0000 - val_tp: 78.0000


Epoch 29/30
Epoch 29/30


2/2 - 0s - 61ms/step - fn: 6.0000 - fp: 111.0000 - loss: 2.9285e-05 - precision: 0.7789 - recall: 0.9849 - tn: 3492.0000 - tp: 391.0000 - val_fn: 5.0000 - val_fp: 19.0000 - val_loss: 0.0714 - val_precision: 0.8041 - val_recall: 0.9398 - val_tn: 898.0000 - val_tp: 78.0000
2/2 - 0s - 61ms/step - fn: 6.0000 - fp: 111.0000 - loss: 2.9285e-05 - precision: 0.7789 - recall: 0.9849 - tn: 3492.0000 - tp: 391.0000 - val_fn: 5.0000 - val_fp: 19.0000 - val_loss: 0.0714 - val_precision: 0.8041 - val_recall: 0.9398 - val_tn: 898.0000 - val_tp: 78.0000


Epoch 30/30
Epoch 30/30


2/2 - 0s - 78ms/step - fn: 9.0000 - fp: 59.0000 - loss: 2.5526e-05 - precision: 0.8680 - recall: 0.9773 - tn: 3544.0000 - tp: 388.0000 - val_fn: 7.0000 - val_fp: 13.0000 - val_loss: 0.0632 - val_precision: 0.8539 - val_recall: 0.9157 - val_tn: 904.0000 - val_tp: 76.0000
2/2 - 0s - 78ms/step - fn: 9.0000 - fp: 59.0000 - loss: 2.5526e-05 - precision: 0.8680 - recall: 0.9773 - tn: 3544.0000 - tp: 388.0000 - val_fn: 7.0000 - val_fp: 13.0000 - val_loss: 0.0632 - val_precision: 0.8539 - val_recall: 0.9157 - val_tn: 904.0000 - val_tp: 76.0000


Using class_weight tells the model that misclassifying a minority-class instance (e.g. a customer who would have accepted a loan) is more costly than misclassifying a majority-class one — which aligns well with business priorities.

## Summary
This project demonstrates the business value of deep learning by applying it to a real-world financial decision: predicting whether a customer will accept a personal loan offer. Using modern Keras practices, we build a neural network that can learn complex, nonlinear relationships between customer attributes (like income, credit card usage, and mortgage) and their likelihood of accepting a loan. This insight enables banks and financial institutions to target high-potential customers, reduce wasted marketing spend, and increase conversion rates—all while minimizing the risk of overlooking valuable leads. Deep learning adds value over traditional models by handling high-dimensional data, uncovering subtle patterns, and adapting quickly as new data becomes available. When used responsibly and explainably, it becomes a powerful tool for customer relationship management, personalized marketing, and strategic decision-making, making it a cornerstone of modern business analytics.